In [8]:
import pandas as pd
import numpy as np
import os
import io
from datetime import datetime, timedelta
from source.utils import CO_MOLECULAR_WEIGHT, NO2_MOLECULAR_WEIGHT, O3_MOLECULAR_WEIGHT, SO2_MOLECULAR_WEIGHT, dfMonthWindows, dfDailyWindows, dfYearWindows, ppb_to_ug_per_m3, ppm_to_ug_per_m3, tryFillMissing
DB_PATH = 'datasets/ontario/'

# granularity = 'months'
granularity = 'daily'
fill_missing = True
max_missing = 0.1

In [9]:
years = ['2020']
# pollutants = ['NO', 'NOx', 'NO2', 'SO2', 'CO', 'O3', 'PM25']
pollutants = ['PM25', ]



windows_map = {}
windows_original_map = {}
for pollutant in pollutants:
    conc_map = {}
    conc_original_map = {}
    aux_map = {}
    for i in range(len(years)):
        with open(os.path.join(DB_PATH, "{}-{}.csv".format(pollutant, years[i])), "r") as f:
            data = f.read()
        print(years[i])
        slices = data.split("\n\n")
        for slice in slices:
            pos = slice.find('Station ID')
            if pos != -1:
                content = slice[pos:]
                content = content.replace(',\n', '\n')
                content = content[:-1]
                df = pd.read_csv(io.StringIO(content), sep=',', header =0)
                station = df['Station ID'][0]
                
            
                hours = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24']
                hoursD = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
                values = np.zeros((365, 24))
                dates = []
                for index, row in df.iterrows():
                    # print('-')
                    # print(index)
                    if index < 365:
                        for k in range(len(hours)):
                            values[index][k] = (row['H{}'.format(hours[k])])
                            if granularity == 'daily':
                                if k == 0:
                                    date = datetime.strptime('{} 00:00:00'.format(row['Date']), '%Y-%m-%d %H:%M:%S')
                                else:
                                    date = date + timedelta(hours=1)
                                if date in dates:
                                    # ! FIX for dates error on csv
                                    dates.append(date + timedelta(days=1))
                                else:
                                    dates.append(date)
                            else:
                                if k == 0:
                                    dateStr = '{}T{}:00:00.000000000'.format(row['Date'], hoursD[k])
                                    date = np.datetime64(dateStr)
                                    # date = pd.to_datetime(dateStr, infer_datetime_format=True)
                                    dates.append(date)
                                    # break
                values[values==9999]=np.nan
                values[values==-999]=np.nan
                
                if granularity != 'daily':
                    # print(values)
                    values = np.mean(values, axis=1)
                    # print(values.shape)
                    if fill_missing:
                        values = tryFillMissing(values,  maxMissing=max_missing)
                else:
                    for t in range(len(values)):
                        if fill_missing:
                            # print(values[t].shape)
                            values[t] = tryFillMissing(values[t], maxMissing=max_missing)
                    values = values.flatten()
                        
                values = values[:len(dates)]
                # print(values)
                dates = np.array(dates)
                
                if  station not in aux_map:
                    aux_map[station] = {}
                
                if pollutant not in aux_map[station]:
                    aux_map[station][pollutant] = (values, dates)
                else:
                    currValues, currDates = aux_map[station][pollutant]
                    aux_map[station][pollutant] = (np.concatenate([currValues, values], axis=0), np.concatenate([currDates, dates], axis=0))
                


2020


In [10]:
# aux_map[47045]['PM25'][0][:24]
# aux_map[47045]['PM25'][1][:24]


array([ 2.,  2.,  2.,  3.,  3.,  2.,  2.,  2.,  3.,  3.,  2.,  2.,  3.,
        3.,  5.,  5.,  6.,  7.,  9., 11., 13., 11., 13., 11.])

5.208333333333333

In [12]:
# station = stations[0]
# values, pol_datetimes = aux_map[station][pollutant]
# df_conc = pd.DataFrame({'date': pol_datetimes, 'value': values})
# df_conc = df_conc.set_index('date')
# # values, dates = dfDailyWindows(df_conc, fill_missing=fill_missing, maxMissing=max_missing)
# values, dates = dfMonthWindows(df_conc, fill_missing=fill_missing, maxMissing=max_missing)
# # values, dates = dfYearWindows(df_conc, fill_missing=fill_missing, maxMissing=max_missing)

In [13]:
values.mean()

nan